<h3>SESSION 7 - 4-axis Scara Robot (Rotary Suction Cup Manipulator)</h3>
Suction cup-type manipulator control, pick & Place of the Scara Robot with orientation control, Inverse Kinematics

The image in the figure represents a 4-Axis SCARA robot and its D-H table (session 9).<br>
<img src="Scara4dofs.jpg" width="500"><br>
From the parameters indicated in the image, we calculate the coordinate transformation matrices for each joint.

In [1]:
import numpy as np
import sympy as sp
from sympy import *
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

# declaramos los símbolos (longitudes, variables, etc.) que se utilizarán para una formulación posterior.


from sympy.physics.mechanics import dynamicsymbols
theta1, theta2, theta3, theta4, lc, la, lb, l4, d3, theta, alpha, a, d = dynamicsymbols('theta1 theta2 theta3 theta4 lc la lb l4 d3 theta alpha a d')
theta1, theta2, theta3, theta4, lc, la, lb, l4, d3, theta, alpha, a, d 

(theta1, theta2, theta3, theta4, lc, la, lb, l4, d3, theta, alpha, a, d)

In [2]:
# The standard homogeneous transformation matrix is represented as:

rot = sp.Matrix([[sp.cos(theta), -sp.sin(theta)*sp.cos(alpha), sp.sin(theta)*sp.sin(alpha)],
                 [sp.sin(theta), sp.cos(theta)*sp.cos(alpha), -sp.cos(theta)*sp.sin(alpha)],
                 [0, sp.sin(alpha), sp.cos(alpha)]])

trans = sp.Matrix([a*sp.cos(theta),a*sp.sin(theta),d])

last_row = sp.Matrix([[0, 0, 0, 1]])

m = sp.Matrix.vstack(sp.Matrix.hstack(rot, trans), last_row)
m

Matrix([
[cos(theta), -sin(theta)*cos(alpha),  sin(alpha)*sin(theta), a*cos(theta)],
[sin(theta),  cos(alpha)*cos(theta), -sin(alpha)*cos(theta), a*sin(theta)],
[         0,             sin(alpha),             cos(alpha),            d],
[         0,                      0,                      0,            1]])

In [3]:
# Transformation: 1st axis from '0' to '1'

m01 = m.subs({ theta:theta1, d:lc, a:la , alpha:0})
N(m01,2)

Matrix([
[cos(theta1), -sin(theta1),   0, la*cos(theta1)],
[sin(theta1),  cos(theta1),   0, la*sin(theta1)],
[          0,            0, 1.0,             lc],
[          0,            0,   0,            1.0]])

In [4]:
# Transformation: 2nd axis from '1' to '2'
m12 = m.subs({ theta:theta2, d:0, a:lb ,alpha:180*np.pi/180})
m12

Matrix([
[cos(theta2),      1.0*sin(theta2),  1.22464679914735e-16*sin(theta2), lb*cos(theta2)],
[sin(theta2),     -1.0*cos(theta2), -1.22464679914735e-16*cos(theta2), lb*sin(theta2)],
[          0, 1.22464679914735e-16,                              -1.0,              0],
[          0,                    0,                                 0,              1]])

In [5]:
# round to 0
m12[0,2]=0
m12[1,2]=0
m12[2,1]=0 #problem when rounding to 0 http://research.iac.es/sieinvens/python-course/source/sympy.html
m12

Matrix([
[cos(theta2),  1.0*sin(theta2),    0, lb*cos(theta2)],
[sin(theta2), -1.0*cos(theta2),    0, lb*sin(theta2)],
[          0,                0, -1.0,              0],
[          0,                0,    0,              1]])

In [6]:
# Transformation: 3rd axis from '2' to '3'
m23 = m.subs({ theta:0, d:d3, a:0 ,alpha:0*np.pi/180})
m23

Matrix([
[1, 0, 0,  0],
[0, 1, 0,  0],
[0, 0, 1, d3],
[0, 0, 0,  1]])

In [7]:
# Transformation: 4th axis from '3' to '4'
m34 = m.subs({ theta:theta4, d:l4, a:0 ,alpha:180*np.pi/180})
m34

Matrix([
[cos(theta4),      1.0*sin(theta4),  1.22464679914735e-16*sin(theta4),  0],
[sin(theta4),     -1.0*cos(theta4), -1.22464679914735e-16*cos(theta4),  0],
[          0, 1.22464679914735e-16,                              -1.0, l4],
[          0,                    0,                                 0,  1]])

In [8]:
# round to 0
m34[0,2]=0
m34[1,2]=0
m34[2,1]=0 #problem when rounding to 0 http://research.iac.es/sieinvens/python-course/source/sympy.html
m34

Matrix([
[cos(theta4),  1.0*sin(theta4),    0,  0],
[sin(theta4), -1.0*cos(theta4),    0,  0],
[          0,                0, -1.0, l4],
[          0,                0,    0,  1]])

In [9]:
# Unsimplified 4-axis Resulting Matrix
m04 = (m01*m12*m23*m34)
m04

Matrix([
[(-sin(theta1)*sin(theta2) + cos(theta1)*cos(theta2))*cos(theta4) + (1.0*sin(theta1)*cos(theta2) + 1.0*sin(theta2)*cos(theta1))*sin(theta4), 1.0*(-sin(theta1)*sin(theta2) + cos(theta1)*cos(theta2))*sin(theta4) - 1.0*(1.0*sin(theta1)*cos(theta2) + 1.0*sin(theta2)*cos(theta1))*cos(theta4),   0, la*cos(theta1) - lb*sin(theta1)*sin(theta2) + lb*cos(theta1)*cos(theta2)],
[ (1.0*sin(theta1)*sin(theta2) - 1.0*cos(theta1)*cos(theta2))*sin(theta4) + (sin(theta1)*cos(theta2) + sin(theta2)*cos(theta1))*cos(theta4), -1.0*(1.0*sin(theta1)*sin(theta2) - 1.0*cos(theta1)*cos(theta2))*cos(theta4) + 1.0*(sin(theta1)*cos(theta2) + sin(theta2)*cos(theta1))*sin(theta4),   0, la*sin(theta1) + lb*sin(theta1)*cos(theta2) + lb*sin(theta2)*cos(theta1)],
[                                                                                                                                         0,                                                                                                                 

In [10]:
# Resulting Matrix we can simplify it further:
mbee= sp.Matrix([[sp.trigsimp(m04[0,0].simplify()), sp.trigsimp(m04[0,1].simplify()), (m04[0,2].simplify()),sp.trigsimp(m04[0,3].simplify())],
                 [sp.trigsimp(m04[1,0].simplify()), sp.trigsimp(m04[1,1].simplify()), (m04[1,2].simplify()),sp.trigsimp(m04[1,3].simplify())],
                 [sp.trigsimp(m04[2,0].simplify()), m04[2,1].simplify(), sp.trigsimp(m04[2,2].simplify()),sp.trigsimp(m04[2,3].simplify())],
                 [m04[3,0].simplify(), m04[3,1].simplify(), m04[3,2].simplify(),m04[3,3].simplify()]])

mbee

Matrix([
[1.0*cos(theta1 + theta2 - theta4), -1.0*sin(theta1 + theta2 - theta4),   0, la*cos(theta1) + lb*cos(theta1 + theta2)],
[1.0*sin(theta1 + theta2 - theta4),  1.0*cos(theta1 + theta2 - theta4),   0, la*sin(theta1) + lb*sin(theta1 + theta2)],
[                                0,                                  0, 1.0,                -1.0*d3 - 1.0*l4 + 1.0*lc],
[                                0,                                  0,   0,                                        1]])

In [12]:
#We substitute the values of la, lb, lc i l4 in the matrix
mbee=mbee.subs({ lc:0.2, la:0.2, lb:0.2, l4:0.0981})
mbee

Matrix([
[1.0*cos(theta1 + theta2 - theta4), -1.0*sin(theta1 + theta2 - theta4),   0, 0.2*cos(theta1 + theta2) + 0.2*cos(theta1)],
[1.0*sin(theta1 + theta2 - theta4),  1.0*cos(theta1 + theta2 - theta4),   0, 0.2*sin(theta1 + theta2) + 0.2*sin(theta1)],
[                                0,                                  0, 1.0,                            0.1019 - 1.0*d3],
[                                0,                                  0,   0,                                          1]])

### COMUNICACIÓN CON COPPELIA Y EXTRACCIÓN DE CONTROLADORES

In [13]:
"""
# we import the necessary libraries and establish a connection
import sim         
def connect(port):

    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Connect
    if clientID == 0: print("connected to", port)
    else: print("could not connect")
    return clientID



clientID = connect(19999)
"""

# I've upgraded the code to  new ZeroMQ implementation.
import coppeliasim_zmqremoteapi_client
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

def connect(port=23000):
    try:
        client = RemoteAPIClient('127.0.0.1', port)
        sim = client.getObject('sim') # Careful! function is getObject now (capital O) 
        print(f"Connected to CoppeliaSim on port {port}")
        return sim
    except Exception as e:
        print(f"Could not connect to CoppeliaSim on port {port}: {e}")
        return None
# Connect to the COPPELIA server
sim = connect()
"""
# Handlers
# We require the handlers for the joints, the suction cup and the suction sensor (Allows us to know if the object is close)

retCode,cuboid=sim.simxGetObjectHandle(clientID,'Cuboid',sim.simx_opmode_blocking)
retCode,suction=sim.simxGetObjectHandle(clientID,'suctionPad',sim.simx_opmode_blocking)
retCode,joint1=sim.simxGetObjectHandle(clientID,'Joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'Joint2',sim.simx_opmode_blocking)
retCode,joint3=sim.simxGetObjectHandle(clientID,'Joint3',sim.simx_opmode_blocking)
retCode,joint4=sim.simxGetObjectHandle(clientID,'Joint4',sim.simx_opmode_blocking)

"""

#New ZeroMQ code
cuboid = sim.getObject('/cuboid')
suction = sim.getObject('/suction')
joint1 = sim.getObject('/joint1')
joint2 = sim.getObject('/joint2')
joint3 = sim.getObject('/joint3')
joint4 = sim.getObject('/joint4')

print( cuboid, suction, joint1, joint2, joint3, joint4 )

connected to 19999
24 28 16 17 21 23


<h3> Suction cup control </h3>
Function that allows you to activate or deactivate the Coppelia suction cup from Jupyter. There are instructions in the Coppelia suctionPad scripts that allow remote use.

In [14]:
def setEffector(val):
# function that triggers the end effector remotely
# val is Int with value 0 or 1 to disable or activate the final actuator.
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
        "suctionPad", sim.sim_scripttype_childscript,"setEffector",[val],[],[],"", sim.simx_opmode_blocking)
    return res

### Rotation matrix from the Euler angles

In [15]:
# We define a function to construct the rotation matrix from the Euler angles

def matrixFromEuler(alpha, beta, gamma):
    # theta y alpha in radians
    # d y a in meters
    Ra = sp.Matrix([[1, 0, 0, 0],
                   [0, sp.cos(alpha), -sp.sin(alpha), 0],
                   [0, sp.sin(alpha), sp.cos(alpha), 0],
                   [0, 0, 0, 1]])
    Rb = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta), 0],
                   [0, 1, 0, 0],
                   [-sp.sin(beta), 0, sp.cos(beta), 0],
                   [0, 0, 0, 1]])
    Rc = sp.Matrix([[sp.cos(gamma), -sp.sin(gamma), 0, 0],
                   [sp.sin(gamma), sp.cos(gamma), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    E = Ra*Rb*Rc
    return E

In [16]:
# WE GET THE POSITION AND ORIENTATION OF THE COBOID WE HAVE TO GET
RetCode,posicion=sim.simxGetObjectPosition(clientID,cuboid, -1, sim.simx_opmode_blocking)
RetCode,orientacion=sim.simxGetObjectOrientation(clientID,cuboid, -1, sim.simx_opmode_blocking)
# Degrees of each axis
orientacion[0]=orientacion[0]*180/np.pi
orientacion[1]=orientacion[1]*180/np.pi
orientacion[2]=orientacion[2]*180/np.pi
print (posicion,orientacion)
print(" Piece Position : ", posicion, "m")
orz= N(orientacion[2],2)
print("Orientation of the Piece on the Z axis : ", orz, "degrees")
# Check that the angles of the piece range from -90 to 90º

[0.25, 0.25, 0.009999925270676613] [-3.796000281768879e-08, -9.02471895034824e-08, 24.999977928391147]
 Piece Position :  [0.25, 0.25, 0.009999925270676613] m
Orientation of the Piece on the Z axis :  25. degrees


In [17]:
# We observe that the object is oriented in the Z Axis about 25 degrees
# We build the object matrix with its orientations and positions
mpieza = sp.Matrix([[1, 0, 0,posicion[0]],
               [0, 1, 0, posicion[1]], 
               [0, 0, 1, posicion[2]], 
               [0, 0, 0, 1]])
Objetivo = mpieza*matrixFromEuler(0, 0,orz*np.pi/180) 
#I only pass the Z Axis, the others are close to 0 (in an atropomorphic the Alga, Beta and Gamma should be passed)

# We adjust the grip position of the suction cup
# IMPORTANT: THE PIECE GIVES US A Z POSITION CENTERED ON THE PIECE OF ONE CM--> BUT THE SUCTION CUP HAS TO GET IT 1mm ABOVE
dventosa=0.011 # It will be located in our case at 2.1 cm in Z, being able to pick up the piece
Objetivo[2,3]=Objetivo[2,3]+dventosa
Objetivo=N(Objetivo,5) 
Objetivo

Matrix([
[0.90631, -0.42262,   0,  0.25],
[0.42262,  0.90631,   0,  0.25],
[      0,        0, 1.0, 0.021],
[      0,        0,   0,   1.0]])

In [18]:
# Sympy's nsolve function searches for solutions that
# set an expression equal to zero. If we require mbee = D, then we look for mbee-D = 0
eq=mbee-Objetivo
eq


Matrix([
[1.0*cos(theta1 + theta2 - theta4) - 0.90631, 0.42262 - 1.0*sin(theta1 + theta2 - theta4), 0, 0.2*cos(theta1 + theta2) + 0.2*cos(theta1) - 0.25],
[1.0*sin(theta1 + theta2 - theta4) - 0.42262, 1.0*cos(theta1 + theta2 - theta4) - 0.90631, 0, 0.2*sin(theta1 + theta2) + 0.2*sin(theta1) - 0.25],
[                                          0,                                           0, 0,                        0.080900061750412 - 1.0*d3],
[                                          0,                                           0, 0,                                                 0]])

## We solve using Sympy's nsolve
### We can obtain the angles and distances of the Joints

In [19]:
try:
    q = nsolve(mbee-Objetivo, (theta1,theta2,d3,theta4),(1, 1, 1, 1), prec=5)
except:
    print('no se encontró la solución, es posible que haya infinitas soluciones')
    q = [0, 0, 0, 0]
q
# It is possible that you will not find a solution due to the precision and initial values given, try changing orientation and position


Matrix([
[ 0.2987],
[0.97339],
[ 0.0809],
[0.83576]])

In [20]:
# Degrees of each axis
q[0]=q[0]-round(q[0]/(np.pi*2))*2*np.pi
q[1]=q[1]-round(q[1]/(np.pi*2))*2*np.pi
q[3]=q[3]-round(q[3]/(np.pi*2))*2*np.pi
print(q)
a1=q[0]*180/np.pi
a2=q[1]*180/np.pi
a3=-q[2]  # destination distance
a4=q[3]*180/np.pi
print(a1,a2,a3,a4)

Matrix([[0.29870], [0.97339], [0.080900], [0.83576]])
17.1144408238486 55.7711404366566 -0.080900 47.8855880780249


<img src="0piece.png" width="500"><br>

In [21]:
import time
time.sleep(3)
setEffector(0)
# PUT IN THE DESIRED POINT
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_oneshot)
time.sleep(3)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, a3, sim.simx_opmode_oneshot)
setEffector(1)
time.sleep(1)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, a3+0.08, sim.simx_opmode_oneshot) # SUBO 8cm
time.sleep(1)

In [212]:
# Si quisiera colocar la pieza por ejemplo en el punto x,y,z=0.25,0,0.021
x,y,z=0.39,0,0.021
# Construimos la matriz destino con sus orientaciones y posiciones
destino = sp.Matrix([[1, 0, 0,x],
               [0, 1, 0, y], 
               [0, 0, 1,z], 
               [0, 0, 0, 1]])
PuntoDestino = destino*matrixFromEuler(0, 0, 0) #Solo le paso la del Eje Z, para jugar al dominó puede ser 0 o 90º

# Ajustamos la posición de cogida de la ventosa 
# IMPORTANTE: LA PIEZA NOS DA UNA POSICIÓN Z CENTRADA EN LA PIEZA DE UN CM--> PERO LA VENTOSA A DE COGERLA 1mm POR ENCIM
PuntoDestino=N(PuntoDestino,5)
PuntoDestino

Matrix([
[1.0,   0,   0,  0.39],
[  0, 1.0,   0,     0],
[  0,   0, 1.0, 0.021],
[  0,   0,   0,   1.0]])

In [213]:
try:
    q = nsolve(mbee-PuntoDestino, (theta1,theta2,d3,theta4),(1, 1, 1, 1), prec=5)
except:
    print('solution was not found, it is possible that there are infinite solutions')
    q = [0, 0, 0, 0]
q


Matrix([
[-0.22408],
[ 0.44815],
[  0.0809],
[ 0.22408]])

In [214]:
# Grados de cada eje destino
q[0]=q[0]-round(q[0]/(np.pi*2))*2*np.pi
q[1]=q[1]-round(q[1]/(np.pi*2))*2*np.pi
q[3]=q[3]-round(q[3]/(np.pi*2))*2*np.pi
print(q)
b1=q[0]*180/np.pi
b2=q[1]*180/np.pi
b3=-q[2]  # distancia destino: restará al prismático la distancia máxima
b4=q[3]*180/np.pi
print(b1,b2,b3,b4)

Matrix([[-0.22408], [0.44815], [0.080900], [0.22408]])
-12.8386476914641 25.6772953829282 -0.080900 12.8386476914641


In [215]:
#Correction needed of orientation:
correccion=a1-b1+a2-b2+a4-orz
correccion
# Substraction between origin and destination angles

82.9325216470660

In [216]:
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
time.sleep(2)
retCode = sim.simxSetJointTargetPosition(clientID, joint4,(correccion)*np.pi/180, sim.simx_opmode_oneshot)
time.sleep(2)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, a3+0.002, sim.simx_opmode_oneshot)
time.sleep(1)
setEffector(0)
time.sleep(1)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, a3+0.08, sim.simx_opmode_oneshot)

<img src="1piece.png" width="500"><br>

<h3>EXERCISE -
    You have to do the programming to place the 3 dominoes together with a proper orientation.
    
Note: It is not necessary to apply image recognition algorithms or the values of the chips, we just want to place the chips in an orderly manner </h3>